![SAR, InSAR, PolSAR, and banner](https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/blackboard-banner.png)

## **Sentinel-1 Radar Forest Degradation Index (RFDI) vs. Landsat NDVI**

### **Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, [Earth Big Data, LLC](http://earthbigdata.com/)**

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/UAFLogo_A_647.png" width="170" align="right" />

This notebook compares C-band Sentinel-1-derived **modified Radar Forest Degradation Index (mRFDI)** maps to Landsat-8 **Normalized Difference Vegetation Index (NDVI)** information over a forested site in the Hindu Kush Himalaya region. It showcases similarities and differences between the two vegetation metrics and shows seasonal dependencies of both the radar (Sentinel-1) and Landsat-8 data for this area.
    
The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. 

**This notebook covers the following data analysis concepts:**

- How to combine SAR and optical data into consistent time series
- How do derive RFDI maps from Sentinel-1 SAR observations
- How to compare RFID and Landsat-8-derived NDVI maps

<hr>

# 0. Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:

1. **[Pandas](https://pandas.pydata.org/)** is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality.
1. **[GDAL](https://www.gdal.org/)** is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
1. **[NumPy](http://www.numpy.org/)** is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. 
1. **[Matplotlib](https://matplotlib.org/index.html)** is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib.  
1. **[SciPY](https://www.scipy.org/about.html)** is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. 

Our first step is to **import them:**

In [ ]:
%%capture
# Check Python version:
import sys
pn = sys.version_info[0]

# Importing relevant python packages
from math import ceil
from pathlib import Path

import pandas as pd
import numpy as np
from osgeo import gdal # for Info
from skimage import exposure # to enhance image display

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()


# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches

from IPython.display import Markdown
from IPython.display import display

font = {'family' : 'monospace',
          'weight' : 'bold',
          'size'   : 18}
plt.rc('font',**font)

plt.rcParams.update({'font.size': 12})


import copy

if pn == 2:
    import cStringIO #needed for the image checkboxes
elif pn == 3:
    import io
    import base64


#---------------------------------------------------------------------------------------------

def moving_window(data, radius=3, verbose=False):
    '''Applies a moving window average to a 2D array and returns the result.
    
    Parameters:
    - data: 2D array of raster values
    - radius: Radius size in pixels
    - verbose: Option to print a message whenever a pixel cannot be added to
        the window
    '''
    [nrow, ncol] = data.shape
    windowed = copy.copy(data)
    for row in range(nrow):
        for col in range(ncol):
            vals_in_window = []
            row_rng = range(row-radius,row+radius+1)
            col_rng = range(col-radius,col+radius+1)
            for y in row_rng:
                for x in col_rng:
                    try:
                        val = data[y][x]
                        vals_in_window.append(val)
                    except Exception as e:
                        if verbose:
                            print("Could not add pixel to list, " +
                                  "may be due to edge effects")
                            print(e)
            windowed[row][col] = np.mean(vals_in_window)
    return(windowed)



#---------------------------------------------------------------------------------------------

# Define a helper function for a 4 part figure with backscatter, NDVI and False Color Infrared
def ebd_plot(bandnbrs, sentinel, landsat):
    fig,ax = plt.subplots(2, 2, figsize=(13,13))
    # Bands for sentinel and landsat:
    # Sentinel VV
    sentinel_vv = img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*sentinel)
    sentinel_vv = 20.*np.log10(sentinel_vv) - 83 # Covert to dB
    # Sentinel VH
    sentinel_vh = img_handle[1].GetRasterBand(bandnbrs[1]).ReadAsArray(*sentinel)
    sentinel_vh = 20.*np.log10(sentinel_vh) - 83 # Covert to dB
    # # Landsat False Color InfraRed
    r = img_handle[5].GetRasterBand(bandnbrs[2]).ReadAsArray(*landsat) / 10000.
    g = img_handle[4].GetRasterBand(bandnbrs[2]).ReadAsArray(*landsat) / 10000.
    b = img_handle[3].GetRasterBand(bandnbrs[2]).ReadAsArray(*landsat) / 10000.
    fcir = np.dstack((r,g,b))
    for i in range(fcir.shape[2]):
        fcir[:,:,i] = exposure.\
        equalize_hist(fcir[:,:,i],
        mask = ~np.equal(fcir[:,:,i], -.9999))
    # Landsat NDVI
    landsat_ndvi = img_handle[2].GetRasterBand(bandnbrs[1]).ReadAsArray(*landsat)
    mask=landsat_ndvi == -9999
    landsat_ndvi = landsat_ndvi / 10000. # Scale to real NDVI value
    landsat_ndvi[mask] = np.nan
    svv = ax[0][0].imshow(sentinel_vv, cmap='gray', vmin=np.nanpercentile(sentinel_vv,5),
                   vmax=np.nanpercentile(sentinel_vv,95))
    cb = fig.colorbar(svv,ax=ax[0][0], orientation='horizontal')
    cb.ax.set_title('C-VV $\gamma^o$ [dB]')
    svh = ax[0][1].imshow(sentinel_vh, cmap='gray', vmin=np.nanpercentile(sentinel_vh,5),
                   vmax=np.nanpercentile(sentinel_vh,95))
    cb = fig.colorbar(svh, ax=ax[0][1], orientation='horizontal')
    cb.ax.set_title('C-VH $\gamma^o$ [dB]')

    nvmin = np.nanpercentile(landsat_ndvi,5)
    nvmax = np.nanpercentile(landsat_ndvi,95)
    # nvmin=-1
    # nvmax=1
    nax = ax[1][0].imshow(landsat_ndvi, cmap='jet', vmin=nvmin,
                   vmax=nvmax)
    cb = fig.colorbar(nax, ax=ax[1][0], orientation='horizontal')
    cb.ax.set_title('NDVI')

    fc = ax[1][1].imshow(fcir)
    # cb = fig.colorbar(fc,cmap=cm.gray,ax=ax[1][1],orientation='horizontal')
    # cb.ax.set_title('False Color Infrared')

    ax[0][0].axis('off')
    ax[0][1].axis('off')
    ax[1][0].axis('off')
    ax[1][1].axis('off')
    ax[0][0].set_title('Sentinel-1 C-VV {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[0][1].set_title('Sentinel-1 C-VH {}'.format(stindex[1][bandnbrs[1]-1].date()))
    ax[1][0].set_title('Landsat-8 NDVI {}'.format(ltindex[bandnbrs[2]-1].date()))
    ax[1][1].set_title('Landsat-8 False Color IR {}'.format(ltindex[bandnbrs[2]-1].date()))
    _ = fig.suptitle('Sentinel-1 Backscatter and Landsat NDVI and False Color IR', size=16)
    
    
# Define a helper function for a 4 part figure with backscatter, mRFDI, and NDVI
def RFDI_NDVI_plot(bandnbrs, sentinel, landsat):
    fig,ax=plt.subplots(2, 2, figsize=(13,13))
    # Bands for sentinel and landsat:
    # Sentinel VV
    sentinel_vv = img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*sentinel)
    sentinel_vvdB = 20. * np.log10(sentinel_vv) - 83 # Covert to dB
    sentinel_vv = np.power(10., sentinel_vvdB/10.) # Covert to power    
    # Sentinel VH
    sentinel_vh = img_handle[1].GetRasterBand(bandnbrs[1]).ReadAsArray(*sentinel)
    sentinel_vhdB = 20.*np.log10(sentinel_vh) - 83 # Covert to dB
    sentinel_vh = np.power(10., sentinel_vhdB/10.) # Covert to power
    # # Sentinel-1 RFDI
    s1_rfdi = (sentinel_vv - sentinel_vh) / (sentinel_vv + sentinel_vh)
    s1_rfdi = moving_window(s1_rfdi, radius=3)
    # Landsat NDVI
    landsat_ndvi = img_handle[2].GetRasterBand(bandnbrs[1]).ReadAsArray(*landsat)
    mask = landsat_ndvi == -9999
    landsat_ndvi = landsat_ndvi / 10000. # Scale to real NDVI value
    landsat_ndvi[mask] = np.nan
    
    svv = ax[0][0].imshow(sentinel_vv, cmap='gray', 
                          vmin=np.nanpercentile(sentinel_vv,5),
                          vmax=np.nanpercentile(sentinel_vv,95))
    cb = fig.colorbar(svv, ax=ax[0][0], orientation='horizontal')
    cb.ax.set_title('C-VV $\gamma^o$ [dB]')
    
    svh = ax[0][1].imshow(sentinel_vh, cmap='gray', vmin=np.nanpercentile(sentinel_vh,5),
                   vmax = np.nanpercentile(sentinel_vh,95))
    cb = fig.colorbar(svh, ax=ax[0][1], orientation='horizontal')
    cb.ax.set_title('C-VH $\gamma^o$ [dB]')

    rfdimin=np.nanpercentile(s1_rfdi, 5)
    rfdimax=np.nanpercentile(s1_rfdi, 95)
    rfdiax = ax[1][0].imshow(s1_rfdi, cmap='jet', vmin=rfdimin,
                   vmax=rfdimax)
    cb = fig.colorbar(rfdiax, ax=ax[1][0], orientation='horizontal')
    cb.ax.set_title('Sentinel-1 mRFDI')
    
    nvmin=np.nanpercentile(landsat_ndvi, 5)
    nvmax=np.nanpercentile(landsat_ndvi, 95)
    # nvmin=-1
    # nvmax=1
    nax = ax[1][1].imshow(landsat_ndvi, cmap='jet', vmin=nvmin,
                   vmax=nvmax)
    cb = fig.colorbar(nax,ax=ax[1][1], orientation='horizontal')
    cb.ax.set_title('NDVI')

    ax[0][0].axis('off')
    ax[0][1].axis('off')
    ax[1][0].axis('off')
    ax[1][1].axis('off')
    ax[0][0].set_title('Sentinel-1 C-VV {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[0][1].set_title('Sentinel-1 C-VH {}'.format(stindex[1][bandnbrs[1]-1].date()))
    ax[1][0].set_title('Sentinel-1 mRFDI {}'.format(stindex[0][bandnbrs[0]-1].date()))
    ax[1][1].set_title('Landsat-8 NDVI {}'.format(ltindex[bandnbrs[2]-1].date()))
    _ = fig.suptitle('Sentinel-1 Backscatter, mRFDI, and Landsat NDVI', size=16)


<hr>

# 1. Load Data Stack

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/Nepalclimate.jpeg" width="400" align="right" />

This notebook will be using Sentinel-1 and Landsat-8 SAR and optical time series information over the site in southern Nepal. The C-band time series includes both VV-polarized (70-image deep stack) and VH-polarized (38 images) data. The Landsat-8 stack is 53 images deep. The C-band data are available to us through the services of the [Alaska Satellite Facility](https://www.asf.alaska.edu/).

Nepal is an interesting site for this analysis due to the significant seasonality of precipitation that is characteristic for this region. Nepal is said to have five seasons: spring, summer, monsoon, autumn and winter. Precipitation is low in the winter (November - March) and peaks dramatically in the summer, with top rain rates in July, August, and September (see figure to the right). As SAR is sensitive to changes in soil moisture, these weather patterns have a noticeable impact on the Radar Cross Section ($\sigma$) time series information. 

---

Before we download anything, **create a working directory for this analysis and change into it:**

In [ ]:
path = Path.cwd()/"data_SAR_optical_comparison"

if not path.exists():
    path.mkdir()

In [ ]:
s3_path = 's3://asf-jupyter-data-west/time_series.zip'
time_series_path = Path(s3_path).name
!aws --region=us-west-2 --no-sign-request s3 cp $s3_path $time_series_path

In [ ]:
if Path(time_series_path).exists():
    asfn.asf_unzip(str(path), time_series_path)
    Path(time_series_path).unlink()

The following lines set path variables needed for data processing. This step is not necessary but it saves a lot of extra typing later. **Define variables for the main data directory as well as for the files containing data and image information:**

In [ ]:
datadirectory = path/'time_series/S32644X696260Y3052060sS1-EBD'
datefile_like = datadirectory/'S32644X696260Y3052060sS1_D_vv_0092_mtfil.dates'
datefile_cross = datadirectory/'S32644X696260Y3052060sS1_D_vh_0092_mtfil.dates'
imagefile_like = datadirectory/'S32644X696260Y3052060sS1_D_vv_0092_mtfil.vrt'
imagefile_cross = datadirectory/'S32644X696260Y3052060sS1_D_vh_0092_mtfil.vrt'
sentinel1_datefile_like = datefile_like
sentinel1_datefile_cross = datefile_cross
sentinel1_imagefile = imagefile_like
sentinel1_imagefile_cross = imagefile_cross
landsat8_ndvi = datadirectory/'landsat/L8_142_041_NDVI.vrt'
landsat8_b3 = datadirectory/'landsat/L8_142_041_B3.vrt'
landsat8_b4 = datadirectory/'landsat/L8_142_041_B4.vrt'
landsat8_b5 = datadirectory/'landsat/L8_142_041_B5.vrt'
landsat8_datefile = datadirectory/'landsat/L8_142_041_NDVI.dates'

---

<div class="alert alert-danger">
<font size="5"><b> Look up Data Set Location on Alaska Satellite Facility's Search Interface</b>:</font> 
    
IF you are curious about the location of this dataset and the vegetation cover in the area, please follow this **[link to ASF's Search Interface](https://search.asf.alaska.edu/#/?zoom=10.400&center=83.010,27.568&polygon=POLYGON((82.9557%2027.5575,83.3227%2027.5575,83.3227%2027.8589,82.9557%2027.8589,82.9557%2027.5575)))**.
</div>

<hr>

# 2. Assess Image Acquisition Dates

Before we start analyzing the available image data, we want to examine the content of our data stack. **First, we read the image acquisition dates for all files in the time series and create a *pandas* date index.**

These are the image **Sentinel-1 SAR** image acquisition dates. We need both the VV and VH polarization channels to calculate the **Radar Forest Degradation index**. You see below that not all acquisition dates offer both polarizations.

In [ ]:
stindex = []
for i in [sentinel1_datefile_like, sentinel1_datefile_cross]:
    sentinell_str = str(i)
    
    sdates = open(sentinell_str).readlines()
    stindex.append(pd.DatetimeIndex(sdates))
    
    j = 1
    print('Bands and dates for',sentinell_str.strip('.dates'))
    for k in stindex[-1]:
        print("{:4d} {}".format(j, k.date()), end=' ')
        j += 1
        if j%5 == 1:
            print()

Next we are plotting the **Landsat-8 optical** image acquisition dates:

In [ ]:
ldates = open(str(landsat8_datefile)).readlines()
ltindex = pd.DatetimeIndex(ldates)
j = 1
print('Bands and dates for', landsat8_ndvi)
for i in ltindex:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

<hr>

# 3. Projection and Georeferencing Information of the SAR and Optical Time Series Data Stacks

For processing of the imagery in this notebook we generate a list of image handles and retrieve projection and georeferencing information. We print out the retrieved information.

In [ ]:
imagelist = [
    sentinel1_imagefile,
    sentinel1_imagefile_cross,
    landsat8_ndvi,
    landsat8_b3,
    landsat8_b4,
    landsat8_b5
]

geotrans = []
proj = []
img_handle = []
xsize = []
ysize = []
bands = []
for i in imagelist:
    i = str(i)
    
    img_handle.append(gdal.Open(i))
    geotrans.append(img_handle[-1].GetGeoTransform())
    proj.append(img_handle[-1].GetProjection())
    xsize.append(img_handle[-1].RasterXSize)
    ysize.append(img_handle[-1].RasterYSize)
    bands.append(img_handle[-1].RasterCount)
# for i in proj:
#     print(i)
# for i in geotrans:
#     print(i)
for i in zip(['C-VV','C-VH','NDVI','B3','B4','B5'],bands,ysize,xsize):
     print(i)

In [ ]:
subset_sentinel = None
# subset_sentinel = (570, 40, 500, 500)  # Adjust or comment out if you don't want a subset
if subset_sentinel == None:
    subset_sentinel = (0, 0, img_handle[0].RasterXSize, img_handle[0].RasterYSize)
    subset_landsat = (0, 0, img_handle[2].RasterXSize, img_handle[2].RasterYSize)
else:
    xoff, yoff, xsize, ysize = subset_sentinel
    xcal = geotrans[0][1] / geotrans[2][1]
    ycal = geotrans[0][5] / geotrans[2][5]
    subset_landsat = (
        int(xoff*xcal),
        int(yoff*ycal),
        int(xsize*xcal),
        int(ysize*ycal)
    )
    
print('Subset Sentinel-1', subset_sentinel, '\nSubset Landsat   ', subset_landsat)

<hr>

# 4. Visualize Sentinel-1 SAR and Landsat-8 NDVI Images for Wet and Dry Seasons

We can pick the bands and plot the Sentinel-1 and Landsat NDVI images of the subset. 

## 4.1 Dry Season Plot

In [ ]:
# DRY SEASON PLOT
bandnbrs=(29, 3, 41)
ebd_plot(bandnbrs, subset_sentinel, subset_landsat)

---

<div class="alert alert-success">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b></font> 

Look at the **dry season** SAR and Optical data example. How would describe the relationship between Landsat-derived NDVI and Sentinel-1 radar brightness in the VV and VH channels?
</div>

<hr>

## 4.2 Wet Season Plot

In [ ]:
# Wet SEASON PLOT
bandnbrs = (52, 20, 53)
ebd_plot(bandnbrs, subset_sentinel, subset_landsat)

---

<div class="alert alert-danger">
<font size="5"><b> Note</b>:</font> 
    
In is much harder to compare SAR and optical data during the wet season as persistent cloud cover obscures the ground regularly for optical data. SAR has a clear advantage observing the ground during rain events.
</div>

<hr>

# 6. Compare the C-Band modified Radar Forest Degradation Index (mRFDI) to the Landsat-8 NDVI

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/RFDI.jpeg" style="padding:5px;" width="430" align="right" />

You have seen in earlier units that we can compute a **Radar Forest Degradation Index (RFDI)** from polarimetric SAR data. The true definition of the RFDI is as follows:

$RFDI = \frac{\gamma^0_{HH} - \gamma^0_{HV}}{\gamma^0_{HH} + \gamma^0_{HV}}$

Here, the terms are all radiometrically corrected imagery. The value of **RFDI** varies between 0 and 1. In general, RFDI can be used to detect both loss of forest cover and its recovery after a disturbance. It is best applied to L-band SAR imagery but can be applied to C-band as well. Approximate guidelines for the interpretation of the **RFDI** are shown on your right.

As you see from the equation above, the **RFDI** in its original form requires *HH* and *HV* channels. For Sentinel-1, these bands are often not available (default mode over land is *VV* and *VH*. Hence, we will work with a slightly modified version (the **mRFDI**) according to:

$mRFDI_{S1} = \frac{\gamma^0_{VV} - \gamma^0_{VH}}{\gamma^0_{VV} + \gamma^0_{VH}}$


---

In the following code cell, we will **plot the *mRFDI* for selected bands**. Please make sure to **pick same-date VV and VH images** to plot the mRFDI and a **simliar-date Landsat-8 scene** for comparison.

<div class="alert alert-danger">
<font size="5"><b> Note</b>:</font> 
    
As it is calculated from C-band data, which does not penetrate extensively into vegetation, the <b>mRFDI</b> often appears a bit noisy. I am applying a $3\times3$ spatial filter to the data to reduce the noise. Hence, <b>the creation of the figure may take a short while. Please be patient.</b>
</div>

In [ ]:
# DRY SEASON mRFDI vs NDVI comparison
bandnbrs=(29, 3, 41)
RFDI_NDVI_plot(bandnbrs, subset_sentinel, subset_landsat)

---

<div class="alert alert-success">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u> </font></b></font>

Please assess the information shown in the figure. While analyzing the figure, please consider the following questions:
- How does the **mRFDI** compare to the **NDVI**?
- Run the cell below which compares **mRFDI** and **NDVI** for a wet season date.
- Compare the **mRFDI** and **NDVI** images for wet and dry season. How does the wet season look different and why?
</div>

---

**Run the following code cell** to generate a mRFDI and NDVI comparison for the wet season in this area of interest.

In [ ]:
# WET SEASON mRFDI & NDVI comparison
bandnbrs=(52, 20, 53)
RFDI_NDVI_plot(bandnbrs, subset_sentinel, subset_landsat)


*edX_RFDIvsNDVI.ipynb - Version 1.0.0 - February 2023*